In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import mysql.connector
from datetime import datetime


In [2]:
load_dotenv()

True

In [3]:
def query_all_table_from_mysql(table_name,db_config):
    conn = mysql.connector.connect(**db_config)    
    cursor = conn.cursor()

    query = f'SELECT * FROM {table_name}'

    cursor.execute(query)

    # Fetch the results into a pandas DataFrame
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

    conn.commit()
    cursor.close()
    conn.close()

    return df

def edited_query_from_table_mysql(db_config,sql_query):
    conn = mysql.connector.connect(**db_config)    
    cursor = conn.cursor()

    query = sql_query

    cursor.execute(query)

    # Fetch the results into a pandas DataFrame
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

    conn.commit()
    cursor.close()
    conn.close()

    return df

def forecast_key_format(row):
    familias_tipo_1 = [278,212,213,225,276,238,270,206,260,261,262,263,275,205,257,258,259] # formato = [cod_fami]
    familias_tipo_2 = [244,245,246,247,202,269,230,252,277,201,264,265,266,267,] # formato = [cod_fami,cod_subg]
    familias_tipo_3 = [239,240,243,] # formato = [cod_fami, cod_subg, cod_dsub]
    familias_tipo_4 = [241, 242, 209] # formato = [cod_fami, cod_grup, cod_subg]
    familias_tipo_5 = [219, 214] # formato = [cod_fami, cod_tipo, cod_subg]
    familias_tipo_6 = [250] # formato = [cod_fami, cod_tipo, cod_grup, cod_subg]
    familias_tipo_7 = [280,251,281,268] # formato = [cod_fami, cod_tipo, cod_grup, cod_subg,cod_marca]
    familias_tipo_8 = [248] # formato = [cod_fami, cod_subg, cod_dsub, cod_tlla]
    familias_tipo_9 = [211] # formato = [cod_fami,cod_grup]

    if int(row['cod_fami']) in familias_tipo_1:
        return row['cod_fami']
    elif int(row['cod_fami']) in familias_tipo_2:
        return row['cod_fami'] + row['cod_subg'] 
    elif int(row['cod_fami']) in familias_tipo_3:
        return row['cod_fami'] + row['cod_subg'] + row['cod_dsub']
    elif int(row['cod_fami']) in familias_tipo_4:
        return row['cod_fami'] + row['cod_grup'] + row['cod_subg']
    elif int(row['cod_fami']) in familias_tipo_5:
        return row['cod_fami'] + row['cod_tipo'] + row['cod_subg']
    elif int(row['cod_fami']) in familias_tipo_6:
        return row['cod_fami'] + row['cod_tipo'] + row['cod_grup'] + row['cod_subg']
    elif int(row['cod_fami']) in familias_tipo_7:
        return row['cod_fami'] + row['cod_tipo'] + row['cod_grup'] + row['cod_subg'] + row['cod_marc']
    elif int(row['cod_fami']) in familias_tipo_8:
        return row['cod_fami'] + row['cod_subg'] + row['cod_dsub'] + row['cod_tlla']
    elif int(row['cod_fami']) in familias_tipo_9:
        return row['cod_fami'] + row['cod_grup']    


In [4]:
#Configurando Datos de Conexión a la DB
db_config = {
    "host": os.getenv("MYSQL_HOST"),
    "user": os.getenv("MYSQL_USER"),
    "password": os.getenv("MYSQL_PASSWORD"),
    "database": os.getenv("MYSQL_DATABASE")
}

In [5]:
# df_store_information = query_all_table_from_mysql('store_information', db_config)
# df_actual_stock = query_all_table_from_mysql('actual_stock', db_config)
# df_warehouse_stock = query_all_table_from_mysql('warehouse_stock', db_config)
df_product_raw = query_all_table_from_mysql("product_information",db_config)
df_product_raw['combination'] = df_product_raw.apply(lambda x: forecast_key_format(x), axis = 1)

In [24]:
today = datetime.now()
year_now = today.year
last_year = year_now - 1
min_date = f'{last_year}-01-01'
max_date = f'{last_year}-12-31'
df_ventas = edited_query_from_table_mysql(db_config,f'SELECT * FROM sales where `Date` between "{min_date}" and "{max_date}"')


In [25]:
df_product_raw = df_product_raw[['ProductId','cod_fami','combination']]
df_last_year_sales = df_ventas.merge(df_product_raw, how = 'left', on = 'ProductId')
df_last_year_sales['Date'] = pd.to_datetime(df_last_year_sales['Date'])
# Group By Week
df_grouped_by_week = df_last_year_sales.groupby(['combination', pd.Grouper(key='Date', freq='W-MON')]).agg({'Quantity': 'sum', 'StoreId':'max','ProductId':'max'}).reset_index() 
df_grouped_by_week.rename(columns = {'Quantity': 'quantity', 'StoreId':'store_id','ProductId':'product_id', 'Date':'date'}, inplace = True)

In [29]:

df_last_year_sales.head()

,codigo_corto,ProductId,StoreId,TicketId,Date,Quantity,PriceTaxesExcluded,SalesPersonId,Origen,cod_fami,combination
0,721957,199980.1001.TM,1013700,L37173777,2023-01-28,1.00,15050.42,9553,POS,251,251348434BV7079
1,1164081,206636.3480.150CM,1011000,L101372066,2023-01-28,2.00,22178.15,9306,POS,239,239BP9PC4
2,834238,209361.1000.UNI,1011000,L101372023,2023-01-28,1.00,6799.16,9318,POS,251,251301462BI1094
3,878783,210189.0005.147CM,1010100,L012853441,2023-01-28,0.80,20161.34,106868,POS,239,239AB8OB4
4,926710,218044.0005.T10,1010100,L012856067,2023-01-28,1.00,45285.71,9597,POS,248,248CF2PV5T10


#### Get All historic Data


In [6]:
min_date = '2018-01-01'
max_date = '2024-12-31'
df_ventas = edited_query_from_table_mysql(db_config,f'SELECT * FROM sales where `Date` between "{min_date}" and "{max_date}"')


In [7]:
df_ventas.head()

,codigo_corto,ProductId,StoreId,TicketId,Date,Quantity,PriceTaxesExcluded,SalesPersonId,Origen
0,721957,199980.1001.TM,1013700,L37173777,2023-01-28,1.00,15050.42,9553,POS
1,1164081,206636.3480.150CM,1011000,L101372066,2023-01-28,2.00,22178.15,9306,POS
2,834238,209361.1000.UNI,1011000,L101372023,2023-01-28,1.00,6799.16,9318,POS
3,878783,210189.0005.147CM,1010100,L012853441,2023-01-28,0.80,20161.34,106868,POS
4,926710,218044.0005.T10,1010100,L012856067,2023-01-28,1.00,45285.71,9597,POS


In [13]:
df_ventas.to_parquet('./historico_ventas_2018_2024_raw.parquet', index = False)

In [15]:
df_product_raw.head()

,ProductId,codigo_corto,cod_fami,des_fami,cod_tipo,des_tipo,cod_grup,des_grup,cod_subg,des_subg,...,cod_fndo,des_fndo,cod_pcon,des_pcon,cod_clor,des_clor,cod_unid,Description,fecha_creacion,combination
0,1.0999.UNI,170315,250,VESTUARIO EXTERIOR,310,Bottom,462,MASCULINO,AZ3,Jeans,...,UNC,Unicolor,MDA,MODA,0999,Generico,UN,JEAN STAFF CHEVI,2023-12-31,250310462AZ3
1,100000.3650.130CM,170316,240,TELAS ALMACEN SI,348,Top,465,MODA,CB8,Caustificados y krinkles,...,UNC,Unicolor,MDA,MODA,3650,Fucsia,MT,LUREX HINDU STR,2023-12-31,240CB8MM3
2,100001.2700.150CM,170317,240,TELAS ALMACEN SI,310,Bottom,455,LINEA,AP5,Crepes,...,UNC,Unicolor,MDA,MODA,2700,Naranja,MT,PH SUPREM EVITA,2023-12-31,240AP5NA
3,100001.4720.150CM,170318,240,TELAS ALMACEN SI,310,Bottom,455,LINEA,AP5,Crepes,...,UNC,Unicolor,BAS,BASICO,4720,Rojo,MT,PH SUPREM EVITA,2023-12-31,240AP5NA
4,100001.7455.150CM,170319,240,TELAS ALMACEN SI,310,Bottom,455,LINEA,AP5,Crepes,...,UNC,Unicolor,MDA,MODA,7455,Azul Aguamarina,MT,PH SUPREM EVITA,2023-12-31,240AP5NA
